In [1]:
import os

import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd

from rasterio.plot import show
from rasterio import features


### Download the data

Download the solar energy potential data for photovoltaic direct normal irradiation from here: https://ezmt.anl.gov/mapexport/energy_potential_solar_pv_v2.zip

Download the solar energy potential data for concentrating direct normal irradiation from here: https://ezmt.anl.gov/mapexport/energy_potential_solar_conc_v2.zip


In [3]:
root_dir = "/Users/d3y010/projects/cerf/suitability"

data_dir_pv = os.path.join(root_dir, "data", "raw", "ezmt_solar", "energy_potential_solar_pvtilt_v2")
data_dir_csp = os.path.join(root_dir, "data", "raw", "ezmt_solar", "energy_potential_solar_conc_v2")

pv_shp = os.path.join(data_dir_pv, "ez_gis.energy_potential_solar_pvtilt_v2.shp")
csp_shp = os.path.join(data_dir_csp, "ez_gis.energy_potential_solar_conc_v2.shp")

template_raster = os.path.join(root_dir, "data", "reference", "cerf_landmask.tif")

temp_dir = os.path.join(root_dir, "validation", "temp")
tech_dir = os.path.join(root_dir, "data", "technology_specific")

output_pv_temp = os.path.join(temp_dir, "pv_temp.tif")
output_pv = os.path.join(tech_dir, "cerf_solar_pv_centralized_potential.tif")

output_csp_temp = os.path.join(temp_dir, "csp_temp.tif")
output_csp = os.path.join(tech_dir, "cerf_solar_csp_centralized_potential.tif")


### Determine irradiance thresholds for suitability

In [4]:
def calc_solar(unit_size_dc, capacity_factor_ac, area_percentage):
    """Calculates the irradiance necessary to produce the AC energy output expected from the given 
    unit size (kW) and AC capacity factor (%)
    
    :param unit_size:             unit size (installed DC capacity) of the solar array in kilowatts
    :type unit_size:              int
    
    :param capacity_factor_ac:    capacity factor (AC) of the solar array
    :type capacity_factor_ac:     float
    
    :param area_percentage:       assumed percentage of square site area used for solar panels
    :type area_percentage:        float
    
    
    :return:                      irradiance required in kWh_m2_day
    
    """
    
    DC_AC_RATIO = 1.3
    HOURS_PER_DAY = 24
    ACRES_PER_MW_AC = 5.8
    ACRES_PER_KM2 = 247.105
    
    kwh_day_ac_max = (unit_size_dc * HOURS_PER_DAY) / DC_AC_RATIO
    kwh_output_actual = kwh_day_ac_max * capacity_factor_ac

    irradiance = kwh_output_actual / unit_size_dc
    irradiance = round(irradiance, 3)
    
    land = ((((unit_size_dc/1000) / DC_AC_RATIO) * ACRES_PER_MW_AC) / ACRES_PER_KM2) / area_percentage
    land = round(land, 2)
    
    print(f'required irradiance level is: {irradiance} kWh/m2/day')
    print(f'required land area is: {land} km-squared')
    
    return irradiance
    


In [23]:
pv_irradiance = calc_solar(unit_size_dc=75000, capacity_factor_ac=0.28, area_percentage=0.75)
csp_irradiance = calc_solar(unit_size_dc=100000, capacity_factor_ac=0.25, area_percentage=0.75)


required irradiance level is: 5.169 kWh/m2/day
required land area is: 1.81 km-squared
required irradiance level is: 4.615 kWh/m2/day
required land area is: 2.41 km-squared


In [18]:
pv = gpd.read_file(pv_shp)

csp = gpd.read_file()


### Create solar potential exclusion layers based on irradiance 

In [19]:
def build_potential_layer(df, template_raster, output_temp, output_file, irradiance, value_field="ann_latilt"):

    with rasterio.open(template_raster) as template:

        metadata = template.meta.copy()

        metadata.update(dtype=np.float64)

        # extract land mask
        land_mask = template.read(1)
        land_mask = np.where(land_mask == 0, np.nan, 1)

        # reproject solar data to GRIDCERF standard
        df = df.to_crs(template.crs)

        with rasterio.open(output_temp, 'w+', **metadata) as out:

            out_arr = out.read(1)

            shapes = ((geom,value) for geom, value in zip(df.geometry, df[value_field]))

            burned = features.rasterize(shapes=shapes, fill=0, out=out_arr, transform=out.transform)

            out.write_band(1, burned)


    with rasterio.open(output_temp) as src:

        metadata = src.meta.copy()

        arr = src.read(1)

        # calculate suitability
        arr = np.where(arr >= irradiance, 0, 1).astype(np.float64)

        # apply land mask
        arr *= land_mask

        with rasterio.open(output_file, 'w', **metadata) as dest:

            dest.write(arr, 1)


In [20]:
build_potential_layer(pv, template_raster, output_pv_temp, output_pv, pv_irradiance, value_field="ann_latilt")



In [21]:
build_potential_layer(csp, template_raster, output_csp_temp, output_csp, csp_irradiance, value_field="ann_dni")

